<a href="https://colab.research.google.com/github/banagg/Kaggle-TSE/blob/master/Tweet_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries imported

In [ ]:
import numpy as np 
import pandas as pd
import json

!pip install tokenizers==0.5.2
!pip install simpletransformers

from simpletransformers.question_answering import QuestionAnsweringModel

Add your saved Kaggle token in the files upload section below

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle
!pip install -q kaggle-cli
!kaggle competitions download -c tweet-sentiment-extraction
!unzip train.csv.zip
!ls

     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 5.3MB 14.3MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 112kB 59.3MB/s 
     |████████████████████████████████| 112kB 64.0MB/s 
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 82.0MB/s]
  0% 0.00/307k [00:00<?, ?B/s]
100% 307k/307k [00:00<00:00, 102MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 41.6MB/s]
Archive:  train.csv.zip
  inflating: train.csv               
kaggle.json  sample_submission.csv  train.csv
sample_data  test.csv		    train.csv.zip


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# sub_df = pd.read_csv('sample_submission.csv')
sub_df = pd.DataFrame(columns=['textID', 'selected_text'])
sub_df['textID'] = list(train_df['textID'][21960:])
!mkdir -p data

train = np.array(train_df)
test = train[21960:]
train = train[:21960]
# test = np.array(test_df)

In [ ]:
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output

qa_train = do_qa_train(train)

with open('data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

In [ ]:
def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile, indent = 1)

In [ ]:
def get_original_text(testStr, subStr):
    testToken = []
    charBreak = '.,!'
    curStr = ''
    strCheck = testStr.lower()
    
    for i in range(0, len(strCheck)):
        if strCheck[i] == ' ' and len(curStr) != 0:
            testToken.append({'str': curStr, 'start': i - len(curStr)})
            curStr = ''
        elif strCheck[i] in charBreak:
            if len(curStr) > 0:
                testToken.append({'str': curStr, 'start': i - len(curStr)})
                curStr = ''
            testToken.append({'str': strCheck[i], 'start': i})
        elif strCheck[i] != ' ':
            curStr += strCheck[i]
    
    if len(curStr) != 0:
        testToken.append({'str': curStr, 'start': i - len(curStr) + 1})

    subToken = []
    curStr = ''
    for i in range(0, len(subStr)):
        if subStr[i] == ' ' and len(curStr) != 0:
            subToken.append(curStr)
            curStr = ''
        elif subStr[i] in charBreak:
            if len(curStr) > 0:
                subToken.append(curStr)
                curStr = ''
            subToken.append(subStr[i])
        elif subStr[i] != ' ':
            curStr += subStr[i]
    
    if len(curStr) != 0:
        subToken.append(curStr)
    
    cnt = 0
    for i in range(0, len(testToken)):
        if testToken[i]['str'] == subToken[cnt]:
            if cnt == 0:
                ans = i
            cnt += 1
        else:
            cnt = 0
            if testToken[i]['str'] == subToken[cnt]:
                ans = i
                cnt += 1
        if cnt >= len(subToken):
            break
    
    startInd = testToken[ans]['start']
    endInd = testToken[ans + len(subToken) - 1]['start'] + len(subToken[-1])
    
    ansStr = testStr[startInd:endInd]
    
    return ansStr

In [ ]:
def length_check(str1, str2, ques):
    if ques == 'positive':
        return False
    ratio = len(str2) / len(str1)
    if (ques == 'neutral' and ratio < 1 and ratio > 0.94) or (ques == 'negative' and ratio < 1 and ratio > 0.91):
        return True
    return False

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel

model = QuestionAnsweringModel('distilbert', 
                               'distilbert-base-uncased-distilled-squad', 
                               args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    },
                              use_cuda=True)

model.train_model('data/train.json')

100%|██████████| 21959/21959 [00:26<00:00, 838.05it/s]


Running loss: 2.814143

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.691472


Running loss: 0.833212


Running loss: 0.772044



In [ ]:
predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)

sub_df['selected_text'] = predictions_df['answer']

for i in range(0, len(sub_df['selected_text'])):
    try:
        sub_df['selected_text'][i] = get_original_text(test[i][1], sub_df['selected_text'][i].lower())
    except Exception:
        pass

    if length_check(test[i][1].strip(), sub_df['selected_text'][i], test[i][-1]):
        sub_df['selected_text'][i] = test[i][1].strip()

sub_df.to_csv('submission.csv', index=False)

100%|██████████| 5521/5521 [00:05<00:00, 1019.79it/s]


To check the similarity score of the validation results, categorised on the basis of sentiment.

In [ ]:
simd = {'positive': 0, 'negative': 0, 'neutral': 0, 'total': 0}
simc = {'positive': 0, 'negative': 0, 'neutral': 0, 'total': 0}

def jaccard(str1, str2): 
	a = set(str1.lower().split()) 
	b = set(str2.lower().split())
	c = a.intersection(b)
	return float(len(c)) / (len(a) + len(b) - len(c))

for i in range(0, len(test)):
	val = jaccard(test[i][2], sub_df['selected_text'][i])
	simd[test[i][3]] += val
	simc[test[i][3]] += 1
	simd['total'] += val
	simc['total'] += 1

for key in simd.keys():
	print(key, simd[key] / simc[key])

positive 0.5006376358765963
negative 0.5158188255709868
neutral 0.9741384037120444
total 0.7000443341633147


In [ ]:
from google.colab import files
files.download('submission.csv') 